In [ ]:
import sqlite3
import logging
from dataclasses import dataclass
from typing import List, Dict, Optional
from odds_calculator import OddsCalculator

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

@dataclass
class Bet:
    event: str
    team: str
    bookmaker: str
    odds: float
    ev: Optional[float] = None
    implied_prob: Optional[float] = None

@dataclass
class ArbitrageOpportunity:
    event: str
    team: str
    odds: List[Dict]
    arbitrage_margin: float
    profit_percentage: float

class BettingAnalyzer:
    def __init__(self, db_path="odds_db.db"):
        self.db_path = db_path

    def fetch_grouped_odds(self) -> Dict:
        """
        Retrieve odds grouped by event and team from the database.
       Returns: Dictionary of events with odds grouped by teams.
        """
##UNDER HERE I DO NOT UNDERSTAND
        try:
            with sqlite3.connect(self.db_path) as connection:
                cursor = connection.cursor()
                fetch_sql = """
                SELECT event, team, bookmaker, odds
                FROM OddsTable

                ORDER BY event, team;
                """
                cursor.execute(fetch_sql)
                rows = cursor.fetchall()
                return self._group_odds_data(rows)

        except sqlite3.Error as e:
            logging.error(f"Database error: {e}")
            return {}
##ABOVE HERE I DO NOT UNDERSTAND


    def _group_odds_data(self, rows) -> Dict:
        """
        Helper function to group odds data by event and team.
        """
        grouped_odds = {}
        for event, team, bookmaker, odds in rows:
            grouped_odds.setdefault(event, {}).setdefault(team, []).append({
                "bookmaker": bookmaker,
                "odds": float(odds)
            })
        return grouped_odds

    def calculate_ev_for_bets(self) -> List[Bet]:
        """
        Calculate Expected Value (EV) for each bet.
        Returns: List of Bet objects with positive EV.
        """
        try:
            grouped_odds = self.fetch_grouped_odds()
            positive_ev_bets = []

            for event, teams in grouped_odds.items():
                for team, odds_data in teams.items():
                    for bet_data in odds_data:
                        try:
                            odds = float(bet_data["odds"])
                            if odds <= 0:
                                continue

                            bookmaker = bet_data["bookmaker"]
                            implied_prob = OddsCalculator.calculate_implied_probability(odds)
                            ev = (implied_prob * odds) - 1

                            if ev > 0:
                                bet = Bet(
                                    event=event,
                                    team=team,
                                    bookmaker=bookmaker,
                                    odds=odds,
                                    ev=round(ev, 4),
                                    implied_prob=round(implied_prob, 4)
                                )
                                positive_ev_bets.append(bet)

                        except ValueError as e:
                            logging.warning(f"Invalid odds value for {event}-{team}: {e}")
                            continue

            return positive_ev_bets

        except Exception as e:
            logging.error(f"Error in EV calculation: {e}")
            return []

    def detect_arbitrage(self) -> List[ArbitrageOpportunity]:
        """
        Detect arbitrage opportunities across bookmakers.
        Returns: List of ArbitrageOpportunity objects.
        """
        try:
            grouped_odds = self.fetch_grouped_odds()
            arbitrage_opportunities = []

            for event, teams in grouped_odds.items():
                for team, odds_data in teams.items():
                    try:
                        valid_odds = [bet["odds"] for bet in odds_data if bet["odds"] > 0]
                        if not valid_odds:
                            continue

                        inverse_sum = sum(1 / odds for odds in valid_odds)
                        margin = 1 - inverse_sum

                        if margin > 0:  # Arbitrage opportunity exists
                            arb_opp = ArbitrageOpportunity(
                                event=event,
                                team=team,
                                odds=odds_data,
                                arbitrage_margin=round(margin, 4),
                                profit_percentage=round(margin * 100, 2)
                            )
                            arbitrage_opportunities.append(arb_opp)

                    except ZeroDivisionError:
                        logging.warning(f"Invalid odds detected for {event}-{team}")
                        continue

            return arbitrage_opportunities

        except Exception as e:
            logging.error(f"Error in arbitrage detection: {e}")
            return []


    def format_bet_output(bet: Bet) -> str:
        """Helper function to format bet output"""
            return (f"Event: {bet.event}, Team: {bet.team}, "
                f"Bookmaker: {bet.bookmaker}, Odds: {bet.odds}, "
                f"EV: {bet.ev}, Implied Prob: {bet.implied_prob}")

    def format_arbitrage_output(arb: ArbitrageOpportunity) -> str:
        """Helper function to format arbitrage output"""
            return (f"Event: {arb.event}, Team: {arb.team}, "
                f"Margin: {arb.arbitrage_margin}, "
                f"Profit %: {arb.profit_percentage}%")

if __name__ == "__main__":
    analyzer = BettingAnalyzer()

    # Test Positive EV Bets
    print("\n=== Positive EV Bets ===")
    positive_ev_bets = analyzer.calculate_ev_for_bets()
    if positive_ev_bets:
        for bet in positive_ev_bets:
            print(format_bet_output(bet))
    else:
        print("No positive EV bets found.")

    # Test Arbitrage Detection
    print("\n=== Arbitrage Opportunities ===")
    arbitrage_opps = analyzer.detect_arbitrage()
    if arbitrage_opps:
        for arb in arbitrage_opps:
            print(format_arbitrage_output(arb))
    else:
        print("No arbitrage opportunities found.")

In [ ]:
# results_storage.py
import sqlite3
import json
import logging
from typing import List, Dict
from dataclasses import dataclass

class ResultsStorage:
    def __init__(self, db_path="odds_db.db"):
        self.db_path = db_path
        self._create_results_tables()

    def _create_results_tables(self):
        """Create tables for storing EV and arbitrage results if they don't exist."""
        try:
            with sqlite3.connect(self.db_path) as connection:
                cursor = connection.cursor()

                # Create table for positive EV bets
                cursor.execute("""
                CREATE TABLE IF NOT EXISTS PositiveEVBets (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    event TEXT NOT NULL,
                    team TEXT NOT NULL,
                    bookmaker TEXT NOT NULL,
                    odds REAL NOT NULL,
                    ev REAL NOT NULL,
                    implied_prob REAL NOT NULL,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
                )
                """)

                # Create table for arbitrage opportunities
                cursor.execute("""
                CREATE TABLE IF NOT EXISTS ArbitrageOpportunities (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    event TEXT NOT NULL,
                    team TEXT NOT NULL,
                    odds_data TEXT NOT NULL,
                    arbitrage_margin REAL NOT NULL,
                    profit_percentage REAL NOT NULL,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
                )
                """)

                connection.commit()
                logging.info("Results tables created successfully")

        except sqlite3.Error as e:
            logging.error(f"Error creating results tables: {e}")

    def save_positive_ev_bets(self, bets: List['Bet']) -> bool:
        """Save positive EV bets to database."""
        try:
            with sqlite3.connect(self.db_path) as connection:
                cursor = connection.cursor()

                insert_sql = """
                INSERT INTO PositiveEVBets
                (event, team, bookmaker, odds, ev, implied_prob)
                VALUES (?, ?, ?, ?, ?, ?)
                """

                bet_data = [(
                    bet.event,
                    bet.team,
                    bet.bookmaker,
                    bet.odds,
                    bet.ev,
                    bet.implied_prob
                ) for bet in bets]

                cursor.executemany(insert_sql, bet_data)
                connection.commit()

                logging.info(f"Saved {len(bets)} positive EV bets")
                return True

        except sqlite3.Error as e:
            logging.error(f"Error saving positive EV bets: {e}")
            return False

    def save_arbitrage_opportunities(self, opportunities: List['ArbitrageOpportunity']) -> bool:
        """Save arbitrage opportunities to database."""
        try:
            with sqlite3.connect(self.db_path) as connection:
                cursor = connection.cursor()

                insert_sql = """
                INSERT INTO ArbitrageOpportunities
                (event, team, odds_data, arbitrage_margin, profit_percentage)
                VALUES (?, ?, ?, ?, ?)
                """

                arb_data = [(
                    arb.event,
                    arb.team,
                    json.dumps(arb.odds),
                    arb.arbitrage_margin,
                    arb.profit_percentage
                ) for arb in opportunities]

                cursor.executemany(insert_sql, arb_data)
                connection.commit()

                logging.info(f"Saved {len(opportunities)} arbitrage opportunities")
                return True

        except sqlite3.Error as e:
            logging.error(f"Error saving arbitrage opportunities: {e}")
            return False

    def get_saved_ev_bets(self, limit: int = 100) -> List[Dict]:
        """Retrieve saved positive EV bets."""
        try:
            with sqlite3.connect(self.db_path) as connection:
                cursor = connection.cursor()
                cursor.execute("""
                    SELECT * FROM PositiveEVBets
                    ORDER BY timestamp DESC
                    LIMIT ?
                """, (limit,))

                columns = [description[0] for description in cursor.description]
                return [dict(zip(columns, row)) for row in cursor.fetchall()]

        except sqlite3.Error as e:
            logging.error(f"Error retrieving saved EV bets: {e}")
            return []

    def get_saved_arbitrage_opportunities(self, limit: int = 100) -> List[Dict]:
        """Retrieve saved arbitrage opportunities."""
        try:
            with sqlite3.connect(self.db_path) as connection:
                cursor = connection.cursor()
                cursor.execute("""
                    SELECT * FROM ArbitrageOpportunities
                    ORDER BY timestamp DESC
                    LIMIT ?
                """, (limit,))

                columns = [description[0] for description in cursor.description]
                return [dict(zip(columns, row)) for row in cursor.fetchall()]

        except sqlite3.Error as e:
            logging.error(f"Error retrieving saved arbitrage opportunities: {e}")
            return []

In [ ]:
##New Tests to be added to main
# Test Functions for Phase 4

from phase_4_ev_arbitrage import fetch_grouped_odds, calculate_ev_for_bets, detect_arbitrage

def test_fetch_grouped_odds():
    grouped_odds = fetch_grouped_odds()
    assert isinstance(grouped_odds, dict), "Grouped odds should be a dictionary."
    assert len(grouped_odds) > 0, "Grouped odds should not be empty."
    print("fetch_grouped_odds() passed.")

def test_calculate_ev_for_bets():
    ev_bets = calculate_ev_for_bets()
    assert isinstance(ev_bets, list), "EV bets should be a list."
    for bet in ev_bets:
        assert "ev" in bet, "Each bet should have an EV value."
        assert bet["ev"] > 0, "EV should be positive for all returned bets."
    print("calculate_ev_for_bets() passed.")

def test_detect_arbitrage():
    arbitrage_opps = detect_arbitrage()
    assert isinstance(arbitrage_opps, list), "Arbitrage opportunities should be a list."
    for opp in arbitrage_opps:
        assert "arbitrage_margin" in opp, "Each opportunity should have an arbitrage margin."
        assert opp["arbitrage_margin"] > 0, "Arbitrage margin should be positive."
    print("detect_arbitrage() passed.")

# Run Tests
if __name__ == "__main__":
    test_fetch_grouped_odds()
    test_calculate_ev_for_bets()
    test_detect_arbitrage()
